In [14]:
import os
import logging
import pandas as pd
import numpy as np
from questions import answer_question
from telegram import Update
from telegram.ext import ContextTypes, CommandHandler, ApplicationBuilder, MessageHandler, filters
from openai import OpenAI
import asyncio
import nest_asyncio
import requests

In [15]:
from functions import functions, run_function
import json

In [16]:
CODE_PROMPT = """
Here are two input:output examples for code generation. Please use these and follow the styling for future requests that you think are pertinent to the request.
Make sure All HTML is generated with the JSX flavoring.
// SAMPLE 1
// A Blue Box with 3 yellow cirles inside of it that have a red outline
<div style={{   backgroundColor: 'blue',
  padding: '20px',
  display: 'flex',
  justifyContent: 'space-around',
  alignItems: 'center',
  width: '300px',
  height: '100px', }}>
  <div style={{     backgroundColor: 'yellow',
    borderRadius: '50%',
    width: '50px',
    height: '50px',
    border: '2px solid red'
  }}></div>
  <div style={{     backgroundColor: 'yellow',
    borderRadius: '50%',
    width: '50px',
    height: '50px',
    border: '2px solid red'
  }}></div>
  <div style={{     backgroundColor: 'yellow',
    borderRadius: '50%',
    width: '50px',
    height: '50px',
    border: '2px solid red'
  }}></div>
</div>
""" 

In [17]:
nest_asyncio.apply()

In [18]:
df = pd.read_csv('processed/embeddings.csv', index_col=0)
df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)

In [19]:
openai = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
tg_bot_token = os.environ['TG_BOT_TOKEN']

In [20]:
messages = [{
    "role":"system",
    "content":"You are a helpful assistant that answers questions."
},{
    "role":"system",
    "content":CODE_PROMPT
}]

In [21]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - $(levelname)s - %(message)s',
    level=logging.INFO
)

In [22]:
async def chat(update: Update, context: ContextTypes.DEFAULT_TYPE):
  messages.append({"role": "user", "content": update.message.text})
  initial_response = openai.chat.completions.create(model="gpt-3.5-turbo",
                                                    messages=messages,
                                                    tools=functions)
  initial_response_message = initial_response.choices[0].message
  messages.append(initial_response_message)
  final_response = None
  tool_calls = initial_response_message.tool_calls
    
  if tool_calls:
    for tool_call in tool_calls:
      name = tool_call.function.name
      args = json.loads(tool_call.function.arguments)
      response = run_function(name, args)
      print(tool_calls)
      messages.append({
          "tool_call_id": tool_call.id,
          "role": "tool",
          "name": name,
          "content": str(response),
      })
      if name == 'svg_to_png_bytes':
        await context.bot.send_photo(chat_id=update.effective_chat.id,
                                     photo=response)
      # Generate the final response
      final_response = openai.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=messages,
      )
      final_answer = final_response.choices[0].message

      # Send the final response if it exists
      if (final_answer):
        messages.append(final_answer)
        await context.bot.send_message(chat_id=update.effective_chat.id,
                                       text=final_answer.content)
      else:
        # Send an error message if something went wrong
        await context.bot.send_message(
            chat_id=update.effective_chat.id,
            text='something wrong happened, please try again')
  #no functions were execute
  else:
    await context.bot.send_message(chat_id=update.effective_chat.id,
                                   text=initial_response_message.content)

In [23]:
async def mozilla(update: Update, context: ContextTypes.DEFAULT_TYPE):
    answer = answer_question(df, question=update.message.text, debug=True)
    await context.bot.send_message(chat_id=update.effective_chat.id, text=answer)

In [24]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id,
                                  text="I am a bot, please talk to me.")

In [25]:
async def image(update: Update, context: ContextTypes.DEFAULT_TYPE):
    response = openai.images.generate(prompt=update.message.text,
                                    model="dall-e-3",
                                    n=1,
                                    size="1024x1024")
    image_url = response.data[0].url
    image_response = requests.get(image_url)
    await context.bot.send_photo(chat_id=update.effective_chat.id,
                               photo=image_response.content)

In [26]:
async def transcribe_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    # Make sure we have a voice file to transcribe
    voice_id = update.message.voice.file_id
    if voice_id:
        file = await context.bot.get_file(voice_id)
        await file.download_to_drive(f"voice_note_{voice_id}.ogg")
        await update.message.reply_text("Voice note downloaded, transcribing now")
        audio_file = open(f"voice_note_{voice_id}.ogg", "rb")
        transcript = openai.audio.transcriptions.create(
        model="whisper-1", file=audio_file
        )
        messages.append({"role": "user", "content": transcript.text})
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        # Feed it back into the LLM
        response_message = response.choices[0].message
        messages.append(response_message)
        await update.message.reply_text(f"Transcript finished:\n{transcript.text}")
        await context.bot.send_message(chat_id=update.effective_chat.id, text=response_message.content)


In [27]:
# Main function to run the bot
async def main() -> None:
    application = ApplicationBuilder().token(tg_bot_token).build()

    start_handler = CommandHandler('start', start)
    chat_handler = MessageHandler(filters.TEXT & (~filters.COMMAND), chat)
    image_handler = CommandHandler('image', image)
    mozilla_handler = CommandHandler('mozilla', mozilla)
    voice_handler = MessageHandler(filters.VOICE, transcribe_message)

    application.add_handler(start_handler)
    application.add_handler(chat_handler)
    application.add_handler(image_handler)
    application.add_handler(mozilla_handler)
    application.add_handler(voice_handler)

    await application.run_polling()

In [ ]:
# Check if the script is run directly (not imported)
if __name__ == '__main__':
    try:
        asyncio.run(main())
    except RuntimeError as e:
        if str(e) == "asyncio.run() cannot be called from a running event loop":
            loop = asyncio.get_event_loop()
            loop.run_until_complete(main())
        else:
            raise

2024-07-03 19:15:27,486 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getMe "HTTP/1.1 200 OK"
2024-07-03 19:15:27,667 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/deleteWebhook "HTTP/1.1 200 OK"
2024-07-03 19:15:27,670 - apscheduler.scheduler - $(levelname)s - Scheduler started
2024-07-03 19:15:27,671 - telegram.ext.Application - $(levelname)s - Application started
2024-07-03 19:15:38,213 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 19:15:48,433 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1XebcqY8o/getUpdates "HTTP/1.1 200 OK"
2024-07-03 19:15:55,540 - httpx - $(levelname)s - HTTP Request: POST https://api.telegram.org/bot7387544315:AAEuwrT3P6dwQ6emRERUjFrp7a1Xe